# Deep Research Agent

## Imports

In [ ]:
import os
import openai
import json
from dataclasses import dataclass, field
from typing import List
from tavily import TavilyClient
from json.decoder import JSONDecodeError
from pydantic_settings import BaseSettings
from IPython.display import Markdown

### Constructing application Configuration object

Note that we have two LLMs configures, we will be using a reasoning model (DeepSeek-R1) for some of the sub agents while we will be using a regular instruction tuned Llama model (Meta-Llama-3.3-70B-Instruct) for other.

In [ ]:
class Config(BaseSettings):
    SAMBANOVA_API_KEY: str
    SAMBANOVA_BASE_URL: str
    LLM_REASONING: str
    LLM_REGULAR: str
    TAVILY_API_KEY: str

Be sure to have your SAMBANOVA_API_KEY (get it [here](https://fnf.dev/4aVUqro)) and TAVILY_API_KEY (get it [here](https://app.tavily.com/)) exported as environment variables before running the next cell.

In [ ]:
config = Config(SAMBANOVA_API_KEY=os.environ["SAMBANOVA_API_KEY"],
                SAMBANOVA_BASE_URL="https://api.sambanova.ai/v1",
                LLM_REASONING="DeepSeek-R1-Distill-Llama-70B",
                LLM_REGULAR="Meta-Llama-3.3-70B-Instruct",
                TAVILY_API_KEY=os.environ["TAVILY_API_KEY"])

### Data Classes to define the System State

In [ ]:
@dataclass
class Search:
    url: str = ""
    content: str = ""

@dataclass
class Research:
    search_history: List[Search] = field(default_factory=list)
    latest_summary: str = ""
    reflection_iteration: int = 0

@dataclass
class Paragraph:
    title: str = ""
    content: str = ""
    research: Research = field(default_factory=Research)

@dataclass
class State:
    report_title: str = ""
    paragraphs: List[Paragraph] = field(default_factory=list)

### Helper functions for data cleaning

In [ ]:
def remove_reasoning_from_output(output):
    return output.split("</think>")[-1].strip()

def clean_json_tags(text):
    return text.replace("```json\n", "").replace("\n```", "")

def clean_markdown_tags(text):
    return text.replace("```markdown\n", "").replace("\n```", "")

### Search tool and a fuction to update System State with search results

In [ ]:
def tavily_search(query, include_raw_content=True, max_results=3):

    tavily_client = TavilyClient(api_key=config.TAVILY_API_KEY)

    return tavily_client.search(query,
                                include_raw_content=include_raw_content,
                                max_results=max_results)

def update_state_with_search_results(search_results, idx_paragraph, state):
    
    for search_result in search_results["results"]:
        search = Search(url=search_result["url"], content=search_result["raw_content"])
        state.paragraphs[idx_paragraph].research.search_history.append(search)

## Agents

Here we define LLM sub-Agents that will read the System State, perform computation and evolve the state.

### Agent for Report structure creation

In [ ]:
output_schema_report_structure = {
        "type": "array",
        "items": {
            "type": "object",
            "properties": {
                "title": {"type": "string"},
                "content": {"type": "string"}
            }
        }
    }

SYSTEM_PROMPT_REPORT_STRUCTURE = f"""
You are a Deep Research assistan. Given a query, plan a structure for a report and the paragraphs to be included.
Make sure that the ordering of paragraphs makes sense.
Once the outline is created, you will be given tools to search the web and reflect for each of the section separately.
Format the output in json with the following json schema definition:

<OUTPUT JSON SCHEMA>
{json.dumps(output_schema_report_structure, indent=2)}
</OUTPUT JSON SCHEMA>

Title and content properties will be used for deeper research.
Make sure that the output is a json object with an output json schema defined above.
Only return the json object, no explanation or additional text.
"""

In [ ]:
class ReportStructureAgent:

    def __init__(self, query: str):

        self.openai_client = openai.OpenAI(
            api_key=config.SAMBANOVA_API_KEY,
            base_url=config.SAMBANOVA_BASE_URL
        )
        self.query = query

    def run(self) -> str:

        response = self.openai_client.chat.completions.create(
            model=config.LLM_REASONING,
            messages=[{"role": "system", "content": SYSTEM_PROMPT_REPORT_STRUCTURE},
                      {"role":"user","content": self.query}]
        )
        return response.choices[0].message.content

    def mutate_state(self, state: State) -> State:

        report_structure = self.run()
        report_structure = remove_reasoning_from_output(report_structure)
        report_structure = clean_json_tags(report_structure)

        report_structure = json.loads(report_structure)

        for paragraph in report_structure:
            state.paragraphs.append(Paragraph(title=paragraph["title"], content=paragraph["content"]))

        return state

### Agent to figure out the first search query for a given paragraph.

In [ ]:
input_schema_first_search = {
            "type": "object",
            "properties": {
                "title": {"type": "string"},
                "content": {"type": "string"}
            }
        }

output_schema_first_search = {
            "type": "object",
            "properties": {
                "search_query": {"type": "string"},
                "reasoning": {"type": "string"}
            }
        }

SYSTEM_PROMPT_FIRST_SEARCH = f"""
You are a Deep Research assistan. You will be given a paragraph in a report, it's title and expected content in the following json schema definition:

<INPUT JSON SCHEMA>
{json.dumps(input_schema_first_search, indent=2)}
</INPUT JSON SCHEMA>

You can use a web search tool that takes a 'search_query' as parameter.
Your job is to reflect on the topic and provide the most optimal web search query to enrich your current knowledge.
Format the output in json with the following json schema definition:

<OUTPUT JSON SCHEMA>
{json.dumps(output_schema_first_search, indent=2)}
</OUTPUT JSON SCHEMA>

Make sure that the output is a json object with an output json schema defined above.
Only return the json object, no explanation or additional text.
"""

In [ ]:
class FirstSearchAgent:

    def __init__(self):

        self.openai_client = openai.OpenAI(
            api_key=config.SAMBANOVA_API_KEY,
            base_url=config.SAMBANOVA_BASE_URL
        )

    def run(self, message) -> str:

        response = self.openai_client.chat.completions.create(
            model=config.LLM_REGULAR,
            messages=[{"role": "system", "content": SYSTEM_PROMPT_FIRST_SEARCH},
                      {"role":"user","content": message}]
        )

        response = remove_reasoning_from_output(response.choices[0].message.content)
        response = clean_json_tags(response)

        response = json.loads(response)

        return response

### Agent to summarise search results of the first search.

In [ ]:
input_schema_first_summary = {
            "type": "object",
            "properties": {
                "title": {"type": "string"},
                "content": {"type": "string"},
                "search_query": {"type": "string"},
                "search_results": {
                    "type": "array",
                    "items": {"type": "string"}
                }
            }
        }

output_schema_first_summary = {
            "type": "object",
            "properties": {
                "paragraph_latest_state": {"type": "string"}
            }
        }

SYSTEM_PROMPT_FIRST_SUMMARY = f"""
You are a Deep Research assistan. You will be given a search query, search results and the paragraph a report that you are researching following json schema definition:

<INPUT JSON SCHEMA>
{json.dumps(input_schema_first_summary, indent=2)}
</INPUT JSON SCHEMA>

Your job is to write the paragraph as a researcher using the search results to align with the paragraph topic and structure it properly to be included in the report.
Format the output in json with the following json schema definition:

<OUTPUT JSON SCHEMA>
{json.dumps(output_schema_first_summary, indent=2)}
</OUTPUT JSON SCHEMA>

Make sure that the output is a json object with an output json schema defined above.
Only return the json object, no explanation or additional text.
"""

In [ ]:
class FirstSummaryAgent:

    def __init__(self):

        self.openai_client = openai.OpenAI(
            api_key=config.SAMBANOVA_API_KEY,
            base_url=config.SAMBANOVA_BASE_URL
        )

    def run(self, message) -> str:

        response = self.openai_client.chat.completions.create(
            model=config.LLM_REGULAR,
            messages=[{"role": "system", "content": SYSTEM_PROMPT_FIRST_SUMMARY},
                      {"role":"user","content": message}]
        )
        return response.choices[0].message.content

    def mutate_state(self, message: str, idx_paragraph: int, state: State) -> State:

        summary = self.run(message)
        summary = remove_reasoning_from_output(summary)
        summary = clean_json_tags(summary)
        
        try:
            summary = json.loads(summary)
        except JSONDecodeError:
            summary = {"paragraph_latest_state": summary}

        state.paragraphs[idx_paragraph].research.latest_summary = summary["paragraph_latest_state"]

        return state

### Agent to Reflect on the latest state of the paragraph.

In [ ]:
input_schema_reflection = {
            "type": "object",
            "properties": {
                "title": {"type": "string"},
                "content": {"type": "string"},
                "paragraph_latest_state": {"type": "string"}
            }
        }

output_schema_reflection = {
            "type": "object",
            "properties": {
                "search_query": {"type": "string"},
                "reasoning": {"type": "string"}
            }
        }

SYSTEM_PROMPT_REFLECTION = f"""
You are a Deep Research assistan. You are responsible for constructing comprehensife paragraphs for a research report. You will be provided paragraph title and planned content summary, also the latest state of the paragraph that you have already created all in the following json schema definition:

<INPUT JSON SCHEMA>
{json.dumps(input_schema_reflection, indent=2)}
</INPUT JSON SCHEMA>

You can use a web search tool that takes a 'search_query' as parameter.
Your job is to reflect on the current state of the paragraph text and think if you havent missed some critical aspect of the topic and provide the most optimal web search query to enrich the latest state.
Format the output in json with the following json schema definition:

<OUTPUT JSON SCHEMA>
{json.dumps(output_schema_reflection, indent=2)}
</OUTPUT JSON SCHEMA>

Make sure that the output is a json object with an output json schema defined above.
Only return the json object, no explanation or additional text.
"""

In [ ]:
class ReflectionAgent:

    def __init__(self):

        self.openai_client = openai.OpenAI(
            api_key=config.SAMBANOVA_API_KEY,
            base_url=config.SAMBANOVA_BASE_URL
        )

    def run(self, message) -> str:

        response = self.openai_client.chat.completions.create(
            model=config.LLM_REGULAR,
            messages=[{"role": "system", "content": SYSTEM_PROMPT_REFLECTION},
                      {"role":"user","content": message}]
        )

        response = remove_reasoning_from_output(response.choices[0].message.content)
        response = clean_json_tags(response)
        response = json.loads(response)

        return response

### Agent to summarise search results after Reflection.

In [ ]:
input_schema_reflection_summary = {
            "type": "object",
            "properties": {
                "title": {"type": "string"},
                "content": {"type": "string"},
                "search_query": {"type": "string"},
                "search_results": {
                    "type": "array",
                    "items": {"type": "string"}
                },
                "paragraph_latest_state": {"type": "string"}
            }
        }

output_schema_reflection_summary = {
            "type": "object",
            "properties": {
                "updated_paragraph_latest_state": {"type": "string"}
            }
        }

SYSTEM_PROMPT_REFLECTION_SUMMARY = f"""
You are a Deep Research assistan.
You will be given a search query, search results, paragraph title and expected content for the paragraph in a report that you are researching.
You are iterating on the paragraph and the latest state of the paragraph is also provided.
The data will be in the following json schema definition:

<INPUT JSON SCHEMA>
{json.dumps(input_schema_reflection_summary, indent=2)}
</INPUT JSON SCHEMA>

Your job is to enrich the current latest state of the paragraph with the search results considering expected content.
Do not remove key information from the latest state and try to enrich it, only add information that is missing.
Structure the paragraph properly to be included in the report.
Format the output in json with the following json schema definition:

<OUTPUT JSON SCHEMA>
{json.dumps(output_schema_reflection_summary, indent=2)}
</OUTPUT JSON SCHEMA>

Make sure that the output is a json object with an output json schema defined above.
Only return the json object, no explanation or additional text.
"""

In [ ]:
class ReflectionSummaryAgent:

    def __init__(self):

        self.openai_client = openai.OpenAI(
            api_key=config.SAMBANOVA_API_KEY,
            base_url=config.SAMBANOVA_BASE_URL
        )

    def run(self, message) -> str:

        response = self.openai_client.chat.completions.create(
            model=config.LLM_REGULAR,
            messages=[{"role": "system", "content": SYSTEM_PROMPT_REFLECTION_SUMMARY},
                      {"role":"user","content": message}]
        )
        return response.choices[0].message.content

    def mutate_state(self, message: str, idx_paragraph: int, state: State) -> State:

        summary = self.run(message)
        summary = remove_reasoning_from_output(summary)
        summary = clean_json_tags(summary)

        try:
            summary = json.loads(summary)
        except JSONDecodeError:
            summary = {"updated_paragraph_latest_state": summary}

        state.paragraphs[idx_paragraph].research.latest_summary = summary["updated_paragraph_latest_state"]

        return state

### Agent to summarise results and produce the formatted report

In [ ]:
input_schema_report_formatting = {
            "type": "array",
            "items": {
                "type": "object",
                "properties": {
                    "title": {"type": "string"},
                    "paragraph_latest_state": {"type": "string"}
            }
        }
    }

SYSTEM_PROMPT_REPORT_FORMATTING = f"""
You are a Deep Research assistan. You have already performed the research and construted final versions of all paragraphs in the report.
You will get the data in the following json format:

<INPUT JSON SCHEMA>
{json.dumps(input_schema_report_formatting, indent=2)}
</INPUT JSON SCHEMA>

Your job is to format the Report nicely and return it in MarkDown.
If Conclusion paragraph is not present, add it to the end of the report from the latest state of the other paragraphs.
Use titles of the paragraphs to create a title for the report.
"""

In [ ]:
class ReportFormattingAgent:

    def __init__(self):

        self.openai_client = openai.OpenAI(
            api_key=config.SAMBANOVA_API_KEY,
            base_url=config.SAMBANOVA_BASE_URL
        )

    def run(self, message) -> str:

        response = self.openai_client.chat.completions.create(
            model=config.LLM_REASONING,
            messages=[{"role": "system", "content": SYSTEM_PROMPT_REPORT_FORMATTING},
                      {"role":"user","content": message}]
        )
        summary = response.choices[0].message.content
        summary = remove_reasoning_from_output(summary)
        summary = clean_markdown_tags(summary)
        
        return summary

## The Topology of the System

In [ ]:
STATE = State()
QUERY="Tell me something interesting about human species"
NUM_REFLECTIONS = 2
NUM_RESULTS_PER_SEARCH = 3
CAP_SEARCH_LENGTH = 20000

In [ ]:
report_structure_agent = ReportStructureAgent(topic)

_ = report_structure_agent.mutate_state(STATE)

first_search_agent = FirstSearchAgent()
first_summary_agent = FirstSummaryAgent()
reflection_agent = ReflectionAgent()
reflection_summary_agent = ReflectionSummaryAgent()
report_formatting_agent = ReportFormattingAgent()

print(f"Total Number of Paragraphs: {len(STATE.paragraphs)}")

idx = 1

for paragraph in STATE.paragraphs:

    print(f"\nParagraph {idx}: {paragraph.title}")

    idx += 1


################## Iterate through paragraphs ##################

for j in range(len(STATE.paragraphs)):

    print(f"\n\n==============Paragraph: {j+1}==============\n")
    print(f"=============={STATE.paragraphs[j].title}==============\n")

    ################## First Search ##################
    
    message = json.dumps(
        {
            "title": STATE.paragraphs[j].title, 
            "content": STATE.paragraphs[j].content
        }
    )
    
    output = first_search_agent.run(message)
    
    search_results = tavily_search(output["search_query"], max_results=NUM_RESULTS_PER_SEARCH)
    
    _ = update_state_with_search_results(search_results, j, STATE)
    
    ################## First Search Summary ##################
    
    message = {
        "title": STATE.paragraphs[j].title,
        "content": STATE.paragraphs[j].content,
        "search_query": search_results["query"],
        "search_results": [result["raw_content"][0:CAP_SEARCH_LENGTH] for result in search_results["results"] if result["raw_content"]]
    }
    
    
    _ = first_summary_agent.mutate_state(message=json.dumps(message), idx_paragraph=j, state=STATE)
    
    ################## Run NUM_REFLECTIONS Reflection steps ##################
    
    for i in range(NUM_REFLECTIONS):
    
        print(f"Running reflection: {i+1}")

        ################## Reflection Step ##################
    
        message = {"paragraph_latest_state": STATE.paragraphs[j].research.latest_summary,
                "title": STATE.paragraphs[j].title,
                "content": STATE.paragraphs[j].content}
        
        output = reflection_agent.run(message=json.dumps(message))

        ################## Reflection Search ##################
        
        search_results = tavily_search(output["search_query"])
        
        _ = update_state_with_search_results(search_results, j, STATE)

        ################## Reflection Search Summary ##################
        
        message = {
            "title": STATE.paragraphs[j].title,
            "content": STATE.paragraphs[j].content,
            "search_query": search_results["query"],
            "search_results": [result["raw_content"][0:20000] for result in search_results["results"] if result["raw_content"]],
            "paragraph_latest_state": STATE.paragraphs[j].research.latest_summary
        }
        
        _ = reflection_summary_agent.mutate_state(message=json.dumps(message), idx_paragraph=j, state=STATE)

################## Generate Final Report ##################

report_data = [{"title": paragraph.title, "paragraph_latest_state": paragraph.research.latest_summary} for paragraph in STATE.paragraphs]

final_report = report_formatting_agent.run(json.dumps(report_data))

### Render the final Report

In [ ]:
display(Markdown(final_report))